In [2]:
import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [21]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [22]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [23]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

#load daily max temps and gets annual max, then obtains time slices 

tasmax = xr.open_dataset("/data/met/hadUK-grid/dmo/tasmax_hadukgrid_uk_12km_day_19600101-20211231.nc")

maxtmax = tasmax.resample(time = "AS-DEC").max()

hist = maxtmax.sel(time = slice("19601201", "19901130")).mean("time")
ref = maxtmax.sel(time = slice("19901201", "20211231")).mean("time")


ds=ref-hist 
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")
da=ds.tasmax

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [24]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [25]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [26]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [27]:
region_all.to_dataframe(name='value').reset_index().to_csv('observations_maxtmax_1975-2005.csv')

In [ ]:
#new variable- tmas observations import sys; sys.path.append('/data/met/processing/10_methods/')
from regridding import *
import geopandas as gpd

In [28]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [29]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [35]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

#load daily max temps and gets annual max, then obtains time slices 

tas = xr.open_dataset("/data/met/hadUK-grid/dmo/tas_hadukgrid_uk_12km_mon_188401-202112.nc")

ann_tas_mean = tas.resample(time = "1AS-DEC",).mean()


hist = ann_tas_mean.sel(time = slice("18851201", "19151201")).mean("time")
ref = ann_tas_mean.sel(time = slice("19901201", "20211201")).mean("time")


ds=ref-hist 
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")
da=ds.tas

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [36]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [37]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [38]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [39]:
region_all.to_dataframe(name='value').reset_index().to_csv('observations_tas_1900-2005.csv')

In [ ]:
#new variable--- drought duration 

In [24]:
# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [25]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [30]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

#load daily max temps and gets annual max, then obtains time slices 

spei6dd = xr.open_dataset("/data/met/hadUK-grid/ind/spei6dd_12km_hadukgrid_mon.nc")

ann_spei6dd_total = spei6dd.resample(time = "1AS-DEC",).sum()


hist = ann_spei6dd_total.sel(time = slice("18851201", "19151201")).mean("time")
ref = ann_spei6dd_total.sel(time = slice("19901201", "20211201")).mean("time")


ds=ref-hist 
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")
da=ds.spei6dd

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [31]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [32]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [33]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [34]:
region_all.to_dataframe(name='value').reset_index().to_csv('observations_drought duration total change_1900-2005.csv')

In [3]:
#new variable rainfall --

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [5]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

#load daily max temps and gets annual max, then obtains time slices 

rainfall = xr.open_dataset("/data/met/hadUK-grid/dmo/rainfall_hadukgrid_uk_12km_day_18910101-20211231.nc")

ann_pr_mean = rainfall.resample(time = "1AS-DEC",).mean()


hist = ann_pr_mean.sel(time = slice("18851201", "19151201")).mean("time")
ref = ann_pr_mean.sel(time = slice("19901201", "20211201")).mean("time")


ds=ref-hist 
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")
da=ds.rainfall

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [6]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [7]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [8]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [9]:
region_all.to_dataframe(name='value').reset_index().to_csv('observations_rainfall_daily_1900-2005.csv')

In [ ]:
#new variable---- prmax5
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# method to get the nearest value to (x,y) from the gridded data in da (assumes everything in OSGB projection)
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.projection_y_coordinate - y)**2 + (da.projection_x_coordinate - x)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 5.76e8)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["projection_x_coordinate", "projection_y_coordinate"])
   
    return val

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile
sf = geopandas.read_file("/home/georgia/Data/GB/westminster")

#load daily max temps and gets annual max, then obtains time slices 

pr = xr.open_dataset("/data/met/hadUK-grid/dmo/rainfall_hadukgrid_uk_12km_day_18910101-20211231.nc")
pr

In [ ]:
pr5 = pr.rolling(time = 5, center = False).sum()

In [ ]:
maxpr5 = pr5.resample(time = "AS-DEC").max()

In [ ]:
hist = pr5.sel(time = slice("18851201", "19151201")).mean("time")
ref = pr5.sel(time = slice("19901201", "20211201")).mean("time")


ds=ref-hist 
ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "NUMBER")
da=ds.pr

# apply the regionmask to the data and average over the x & y dimensions
region_da = da.where(rm).mean(["projection_x_coordinate", "projection_y_coordinate"])

In [ ]:
# find nearest neighbour

region_nn = xr.concat([nearest_px(*sf.centroid[i].xy, da).expand_dims(region = [sf.NUMBER[i]]).assign_coords(constituency = ("region", [sf.FILE_NAME[i]])) for i in range(len(sf))], "region")

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# combine regionmask with nearest neighbour where regionmask didn't pick anything up

region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")

In [ ]:
region_all=region_all.assign_coords(region=('region',sf.CODE))

In [ ]:
region_all.to_dataframe(name='value').reset_index().to_csv('observations_max5dayprecipitation_1900-2005.csv')